In [1]:
import hjson
import csv

import os
import sys

import pulp
import itertools
import pandas as pd
import scipy as sc
import numpy as np

In [38]:
constr = {}
with open("../config/constraints.hjson",'r',encoding='utf-8') as conf:
    constr = hjson.load(conf)
    # S for string
for conf in list(constr.items()):
    # print(conf)
    # print(val)
    if conf[1] == -1:
#         print(conf)
        constr.pop(conf[0])

#         del conf
print(constr)
# with open()

OrderedDict([('Std_TFe', 58), ('Rls_TFe', 2), ('Std_SiO2', 5), ('Rls_SiO2', 0.3), ('Std_MgO', 1.6), ('Rls_MgO', 0.2), ('Std_Al2O3', 1.55), ('Rls_Al2O3', 0.55), ('Std_K2O', 0.075), ('Rls_K2O', 0.075), ('Std_FeO', 8), ('Rls_FeO', 0.5), ('Std_ZnO', 0.05), ('Rls_ZnO', 0.05), ('Std_CaOSiO2_ratio', 2.1), ('Rls_CaOSiO2_ratio', 0.2)])


In [42]:
with open("../config/mineral.csv",'r',encoding='gb2312') as form:
    data = csv.reader(form)
    for d in data:
        print(d)

['Type', 'Name', 'TFe', 'SiO2', 'Al2O3', 'CaO', 'MgO', 'S', 'P', 'K2O', 'LOI', 'H2O', 'UnitPrice']
['M', '', '', '', '', '', '', '', '', '', '', '', '']
['M', '', '', '', '', '', '', '', '', '', '', '', '']
['M', '', '', '', '', '', '', '', '', '', '', '', '']
['M', '', '', '', '', '', '', '', '', '', '', '', '']
['M', '', '', '', '', '', '', '', '', '', '', '', '']
['M', '', '', '', '', '', '', '', '', '', '', '', '']
['M', '', '', '', '', '', '', '', '', '', '', '', '']
['H', '热返矿', '', '', '', '', '', '', '', '', '', '', '']
['C', '冷返矿', '', '', '', '', '', '', '', '', '', '', '']
['Z', '杂矿', '', '', '', '', '', '', '', '', '', '', '']
['CA', '生石灰粉', '', '', '', '', '', '', '', '', '', '', '']
['CB', '高镁粉', '', '', '', '', '', '', '', '', '', '', '']
['J', '焦粉', '', '', '', '', '', '', '', '', '', '', '']
['M', '煤粉', '', '', '', '', '', '', '', '', '', '', '']


In [ ]:
# Constraints:

## STD
## 含铁合计100
## 脱水核算单位LOI